# LanceDB fundamentals

LanceDB is an open-source vector database designed to handle large-scale vector data efficiently. It provides a robust platform for storing, indexing, and querying high-dimensional vectors, making it ideal for applications in machine learning, computer vision, and natural language processing.

We'll be using LanceDB for the purpose of creating RAG applications. However before we dive into that, let's cover some fundamental concepts of LanceDB, so that you know what is happening under the hood.

## Connect LanceDB

- when connecting to LanceDB it will create a folder for your lancedb database if it does not already exist

In [1]:
import lancedb

db = lancedb.connect(uri = "vector_database")
db

LanceDBConnection(uri='/Users/kidquatro/Documents/STI-DE24/Ai Engineering/ai-engineering-robin-sundman-nilsson-de24/09_lancedb_vector_database/vector_database')

In [2]:
db.uri

'/Users/kidquatro/Documents/STI-DE24/Ai Engineering/ai-engineering-robin-sundman-nilsson-de24/09_lancedb_vector_database/vector_database'

## Create a Table

when creating a table, it is stored as a folder with .lance suffix containing the following folders
- _transactions
- _versions
- data

a table is a pyarraow table stored in the data folder

In [4]:
import json
with open("animals_text_embeddings.json") as file:
    data = json.loads(file.read())

data

[{'text': 'A small brown dog running.', 'vector': [0.12, 0.85, 0.33]},
 {'text': 'A cat resting quietly on a sofa.', 'vector': [0.4, 0.91, 0.1]},
 {'text': 'A large gray elephant drinking water.',
  'vector': [0.88, 0.22, 0.55]},
 {'text': 'A fast cheetah sprinting across the savannah.',
  'vector': [0.95, 0.12, 0.72]},
 {'text': 'A colorful parrot perched on a branch.',
  'vector': [0.25, 0.66, 0.81]},
 {'text': 'A frog sitting on a lily pad.', 'vector': [0.14, 0.44, 0.27]}]

In [8]:
table_animals = db.create_table("animals_text", exist_ok=True, data=data)
table_animals


LanceTable(name='animals_text', version=1, _conn=LanceDBConnection(uri='/Users/kidquatro/Documents/STI-DE24/Ai Engineering/ai-engineering-robin-sundman-nilsson-de24/09_lancedb_vector_database/vector_database'))

In [9]:
# it's a pyarrow table 
table_animals.head()

pyarrow.Table
text: string
vector: fixed_size_list<item: float>[3]
  child 0, item: float
----
text: [["A small brown dog running.","A cat resting quietly on a sofa.","A large gray elephant drinking water.","A fast cheetah sprinting across the savannah.","A colorful parrot perched on a branch."]]
vector: [[[0.12,0.85,0.33],[0.4,0.91,0.1],[0.88,0.22,0.55],[0.95,0.12,0.72],[0.25,0.66,0.81]]]

In [11]:
# converting to pandas 
table_animals.to_pandas()

,text,vector
0,A small brown dog running.,"[0.12, 0.85, 0.33]"
1,A cat resting quietly on a sofa.,"[0.4, 0.91, 0.1]"
2,A large gray elephant drinking water.,"[0.88, 0.22, 0.55]"
3,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]"
4,A colorful parrot perched on a branch.,"[0.25, 0.66, 0.81]"
5,A frog sitting on a lily pad.,"[0.14, 0.44, 0.27]"


add some more data to the table

In [12]:
more_data = [
    {"text": "A panda eating bamboo peacefully.", "vector": [0.51, 0.37, 0.82]},
    {"text": "A lion roaring loudly on a rock.", "vector": [0.93, 0.18, 0.41]},
]

table_animals.add(more_data)

AddResult(version=2)

In [13]:
table_animals.to_pandas()

,text,vector
0,A small brown dog running.,"[0.12, 0.85, 0.33]"
1,A cat resting quietly on a sofa.,"[0.4, 0.91, 0.1]"
2,A large gray elephant drinking water.,"[0.88, 0.22, 0.55]"
3,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]"
4,A colorful parrot perched on a branch.,"[0.25, 0.66, 0.81]"
5,A frog sitting on a lily pad.,"[0.14, 0.44, 0.27]"
6,A panda eating bamboo peacefully.,"[0.51, 0.37, 0.82]"
7,A lion roaring loudly on a rock.,"[0.93, 0.18, 0.41]"


## Create an empty table and then delete it

In [14]:
# LanceModel is a Pydantic model that can be converted to LanceDB table
from lancedb.pydantic import LanceModel

class JokeSchema(LanceModel):
    joke: str 
    rating: int


db.create_table(name = "jokes",schema=JokeSchema, exist_ok=True)

LanceTable(name='jokes', version=1, _conn=LanceDBConnection(uri='/Users/kidquatro/Documents/STI-DE24/Ai Engineering/ai-engineering-robin-sundman-nilsson-de24/09_lancedb_vector_database/vector_database'))

In [15]:
db.table_names()

['animals_text', 'jokes']

In [16]:
db.drop_table("jokes")

In [17]:
db.table_names()

['animals_text']

## Open existing table

In [18]:
# since db is already connected to vector_database it has animals_text as table
db.open_table("animals_text").head()

pyarrow.Table
text: string
vector: fixed_size_list<item: float>[3]
  child 0, item: float
----
text: [["A small brown dog running.","A cat resting quietly on a sofa.","A large gray elephant drinking water.","A fast cheetah sprinting across the savannah.","A colorful parrot perched on a branch."]]
vector: [[[0.12,0.85,0.33],[0.4,0.91,0.1],[0.88,0.22,0.55],[0.95,0.12,0.72],[0.25,0.66,0.81]]]

## Vector search in LanceDB

Uses ANN - approximate nearest neighbours for searching vectors

- search with vector directly
- note that the vector column is already an example embedding simulated with LLM

A more realistic approach is to just have text records, use an existing embedding model and embedd those text records, then embed the query vector and find closest match 


In [19]:
table_animals.to_pandas()

,text,vector
0,A small brown dog running.,"[0.12, 0.85, 0.33]"
1,A cat resting quietly on a sofa.,"[0.4, 0.91, 0.1]"
2,A large gray elephant drinking water.,"[0.88, 0.22, 0.55]"
3,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]"
4,A colorful parrot perched on a branch.,"[0.25, 0.66, 0.81]"
5,A frog sitting on a lily pad.,"[0.14, 0.44, 0.27]"
6,A panda eating bamboo peacefully.,"[0.51, 0.37, 0.82]"
7,A lion roaring loudly on a rock.,"[0.93, 0.18, 0.41]"


In [20]:
query_vector = [0.5, 0.2, 0.9]

# finds closest rows based on the Euclidean distance by default
table_animals.search(query_vector).limit(3).to_pandas()

,text,vector,_distance
0,A panda eating bamboo peacefully.,"[0.51, 0.37, 0.82]",0.0354
1,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]",0.2413
2,A large gray elephant drinking water.,"[0.88, 0.22, 0.55]",0.2673


## Embeddings API

LanceDB provides an Embeddings API that allows you to easily generate and store vector embeddings for your data. This is particularly useful for applications such as semantic search, recommendation systems, and clustering.

We'll use embeddings model from Gemini but, you can use any embedding model of your choice.

Embeddings are vectors that represent the semantic meaning of data, such as text or images. By converting data into embeddings, you can perform similarity searches and other operations more effectively

In [33]:
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry

model = get_registry().get("gemini-text").create(name = "gemini-embedding-001")

class JokeModel(LanceModel):
    joke: str = model.SourceField() # marks text as input to embedding function
    vector: Vector(3072) = model.VectorField() # vector is destination of computed embedding

table_jokes = db.create_table("jokes", schema = JokeModel, exist_ok=True)
table_jokes



LanceTable(name='jokes', version=3, _conn=LanceDBConnection(uri='/Users/kidquatro/Documents/STI-DE24/Ai Engineering/ai-engineering-robin-sundman-nilsson-de24/09_lancedb_vector_database/vector_database'))

In [34]:
import pandas as pd 
with open("jokes.json", "r") as file:
    jokes_data = json.loads(file.read())

df_jokes = pd.DataFrame(jokes_data).rename({"jokes": "joke"}, axis=1)
df_jokes.head()

,joke
0,Parallel lines have so much in common—it’s sad...
1,"ETL stands for “Extract, Transform, Leave for ..."
2,What do you call a snake that runs your script...
3,"Gold walks into a bar. The bartender says, “Au..."
4,C# devs don’t argue; they just throw exceptions.


adds data to our table

In [35]:
table_jokes.add(df_jokes)


AddResult(version=4)

we can see that vector column has been created automatically and embeddings have been generated for each joke

In [36]:
table_jokes.head(2)

pyarrow.Table
joke: string not null
vector: fixed_size_list<item: float>[3072]
  child 0, item: float
----
joke: [["Parallel lines have so much in common—it’s sad they’ll never meet.","ETL stands for “Extract, Transform, Leave for the next person.”"]]
vector: [[[-0.024001757,0.01247358,-0.024144737,-0.06704516,0.017059995,...,0.011402721,-0.017770408,0.011606138,0.0004488597,0.012175956],[-0.015356114,0.0211365,-0.021389864,-0.07957475,0.008829045,...,0.023671096,0.00070166244,0.003873497,0.0006493248,0.01587487]]]

In [37]:
table_jokes.to_pandas().head()

,joke,vector
0,Parallel lines have so much in common—it’s sad...,"[-0.024001757, 0.01247358, -0.024144737, -0.06..."
1,"ETL stands for “Extract, Transform, Leave for ...","[-0.015356114, 0.0211365, -0.021389864, -0.079..."
2,What do you call a snake that runs your script...,"[-0.01761013, 0.0031474787, -0.015632002, -0.0..."
3,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0..."
4,C# devs don’t argue; they just throw exceptions.,"[-0.0068662865, -0.005415149, 0.0044965413, -0..."


In [38]:
table_jokes.search("data engineering jokes").limit(8).to_pandas()

,joke,vector,_distance
0,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.461179
1,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.461179
2,"Data engineer motto: If it works, don’t touch ...","[-0.020296954, 0.020327171, -0.009069326, -0.0...",0.542293
3,"Data engineer motto: If it works, don’t touch ...","[-0.020296954, 0.020327171, -0.009069326, -0.0...",0.542293
4,What do you call a snake that runs your script...,"[-0.01761013, 0.0031474787, -0.015632002, -0.0...",0.655434
5,What do you call a snake that runs your script...,"[-0.01761013, 0.0031474787, -0.015632002, -0.0...",0.655434
6,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.667807
7,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.667807


In [39]:
table_jokes.search("give me jokes on C#").limit(8).to_pandas()

,joke,vector,_distance
0,The C# compiler walked into a bar. The bartend...,"[-0.01868987, 0.018796643, -0.009748903, -0.07...",0.458438
1,The C# compiler walked into a bar. The bartend...,"[-0.01868987, 0.018796643, -0.009748903, -0.07...",0.458438
2,Why did the C# developer go broke? He kept usi...,"[-0.02175711, 0.0073709465, -0.015515881, -0.0...",0.483707
3,Why did the C# developer go broke? He kept usi...,"[-0.02175711, 0.0073709465, -0.015515881, -0.0...",0.483707
4,Why is C# like a musical? It has so many classes.,"[-0.012926053, 0.0034431089, -0.017093537, -0....",0.524282
5,Why is C# like a musical? It has so many classes.,"[-0.012926053, 0.0034431089, -0.017093537, -0....",0.524282
6,C# devs don’t argue; they just throw exceptions.,"[-0.0068662865, -0.005415149, 0.0044965413, -0...",0.614111
7,C# devs don’t argue; they just throw exceptions.,"[-0.0068662865, -0.005415149, 0.0044965413, -0...",0.614111


In [40]:
table_jokes.search("give me chemistry related jokes").limit(8).to_pandas()

,joke,vector,_distance
0,I told a chemistry joke… there was no reaction.,"[-0.022922393, 0.017959604, -0.029222224, -0.0...",0.576598
1,I told a chemistry joke… there was no reaction.,"[-0.022922393, 0.017959604, -0.029222224, -0.0...",0.576598
2,Why did the chemist ground his kids? Because t...,"[-0.023257235, 0.016145445, -0.029016329, -0.0...",0.613355
3,Why did the chemist ground his kids? Because t...,"[-0.023257235, 0.016145445, -0.029016329, -0.0...",0.613355
4,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.671744
5,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.671744
6,Chemists have all the solutions… mostly in bea...,"[-0.012411982, -0.002532549, -0.0061755716, -0...",0.673519
7,Chemists have all the solutions… mostly in bea...,"[-0.012411982, -0.002532549, -0.0061755716, -0...",0.673519


In [41]:
table_jokes.search("give me nature related jokes").limit(8).to_pandas()

,joke,vector,_distance
0,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.592044
1,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.592044
2,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.693084
3,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.693084
4,Why did the chemist ground his kids? Because t...,"[-0.023257235, 0.016145445, -0.029016329, -0.0...",0.723715
5,Why did the chemist ground his kids? Because t...,"[-0.023257235, 0.016145445, -0.029016329, -0.0...",0.723715
6,I told a chemistry joke… there was no reaction.,"[-0.022922393, 0.017959604, -0.029222224, -0.0...",0.725794
7,I told a chemistry joke… there was no reaction.,"[-0.022922393, 0.017959604, -0.029222224, -0.0...",0.725794


## Hybrid search

hybrid search combines traditional keyword-based search with vector-based search to provide more relevant results. This approach leverages the strengths of both methods, allowing for more accurate and context-aware retrieval of information.

In [42]:
# to enable keyword based search on joke column
table_jokes.create_fts_index("joke", replace=True)


In [43]:
from lancedb import rerankers

# reranks the results from a vector and full text search
reranker = rerankers.RRFReranker()

results = table_jokes.search(
    "give me nature related jokes",
    query_type="hybrid",
    vector_column_name="vector",
    fts_columns="joke",
).rerank(reranker).limit(8).to_pandas()

# somewhat different to before
results


,joke,vector,_relevance_score
0,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.032018
1,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.032002
2,I told a chemistry joke… there was no reaction.,"[-0.022922393, 0.017959604, -0.029222224, -0.0...",0.031319
3,I told a chemistry joke… there was no reaction.,"[-0.022922393, 0.017959604, -0.029222224, -0.0...",0.030835
4,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.015873
5,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.015625
6,Why did the chemist ground his kids? Because t...,"[-0.023257235, 0.016145445, -0.029016329, -0.0...",0.015385
7,Why did the chemist ground his kids? Because t...,"[-0.023257235, 0.016145445, -0.029016329, -0.0...",0.015152


## Rule of thumb 

- For exact matching -> FTS
- For meaning based matching -> Vector search
- Unpredictable or mixed queries -> Hybrid search
